<a href="https://colab.research.google.com/github/TonmoyTalukder/Rank-Your-Summaries-Enhancing-Bengali-Text-Summarization-via-Ranking-based-Approach/blob/main/Summary%20Generation%20with%20Pre-trained%20Models/mT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset Loading

## Dataset I

In [ ]:
#install dataset
!pip install datasets

In [ ]:
from datasets import load_dataset
from pprint import pprint

In [ ]:
dataset = load_dataset('csebuetnlp/xlsum', 'bengali', split='train')

In [ ]:
print("Dataset length: ", len(dataset))
print("\n First item dataset[0]: ")
pprint(dataset[1])

In [ ]:
pprint(len(dataset[0]['text']))
pprint(type(dataset[0]['text']))

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset)

df.head()

In [ ]:
df.head(10)

In [ ]:
df['text'][2]

In [ ]:
rows = df.shape[0]
rows

8102

## Dataset II

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/SCDataset/Final/text_summarization.csv')

df.head()

,category,summary,text
0,technology,অ্যাপসে মিলবে ঢাকাসহ তিন জেলা আদালতের তথ্য,"ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙ..."
1,bangladesh,বিজ্ঞান ও প্রকৌশলে মার্কিন সর্বোচ্চ সম্মাননা...,যুক্তরাষ্ট্রে বিজ্ঞান ও প্রকৌশলে পেশা শুরুর প...
2,bangladesh,বিকল্প শিশুখাদ্য গ্রহণে শিশুর মৃত্যু হলে শাস্ত...,বিকল্প শিশুখাদ্য ও বাণিজ্যিকভাবে উত্পাদিত শিশু...
3,bangladesh,ট্রেনে কাটা পড়ে সাবেক সিভিল সার্জনের মৃত্যু,বগুড়ার আদমদীঘির সান্তাহারে গতকাল শনিবার ট্রেন...
4,bangladesh,যাত্রাবাড়ীতে চুলা জ্বালাতে গিয়ে দুই কর্মচারী ...,যাত্রাবাড়ীর একটি রেস্তোরাঁয় গতকাল বুধবার ভোর...


In [ ]:
rows = df.shape[0]
rows

80331

# Install and Imports

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install git+https://github.com/csebuetnlp/normalizer

import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer


from transformers import XLMProphetNetForConditionalGeneration, XLMProphetNetTokenizer


from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AlbertTokenizer, AutoTokenizer

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Functions

In [ ]:
def cleanText(text2clean):
    empty_string = ""

    space = chr(32)
    flag  = 0

    for i in range(len(text2clean)):
        if text2clean[i] != "'" and text2clean[i] != "\n":
            if text2clean[i] == space and flag == 0:
                flag = 1
                empty_string = empty_string + text2clean[i]
            elif text2clean[i] != space:
                empty_string = empty_string + text2clean[i]
                flag = 0
    return empty_string

# Model Loading 1: csebuetnlp/mT5_multilingual_XLSum

### Model Setup

In [ ]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.to(device)

### Run dataset over the Model

In [ ]:
# Create empty DataFrame
genDf = pd.DataFrame(columns = ["title", "text", "givenSummary", "mT5_multilingual_XLSum"])

# Load over the dataset
for i in range(rows):
    # if i < 1000:
    # if i > 999 and i < 2000:
    # if i > 1999 and i < 3000:
    # if i > 2999 and i < 4000:
    # if i > 3999 and i < 5000:
    # if i > 4999 and i < 6000:
    # if i > 5999 and i < 7000:
    # if i > 6999 and i < 8000:
    if i > 7999:
        article_text = normalize(df['text'][i])

        input_ids = tokenizer(
            [WHITESPACE_HANDLER(article_text)],
            return_tensors="pt",
            padding="max_length",
            max_length=512,
            truncation=True
        )["input_ids"]

        output_ids = model.generate(
            input_ids=input_ids.cuda(),
            # input_ids=input_ids,
            max_length=400,
            min_length=64,
            no_repeat_ngram_size=2,
            num_beams=4
        )[0]

        summary = tokenizer.decode(
            output_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )


        # df.loc["Saanvi"] = [96, 90]
        genDf.loc[i] = [df['title'][i], df['text'][i], df['summary'][i], summary]

        print(i, '  ', summary)
        # if i == 10:
        #     break

# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v1000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v2000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v3000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v4000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v5000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v6000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v7000.csv')
# genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v8000.csv')
genDf.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v8KPlus.csv')

## Concat Splitted Dataset

In [ ]:
df0 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v1000.csv")
df1 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v2000.csv")
df2 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v3000.csv")
df3 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v4000.csv")
df4 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v5000.csv")
df5 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v6000.csv")
df6 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v7000.csv")
df7 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v8000.csv")
df8 = pd.read_csv("/content/drive/MyDrive/SCDataset/mT5_multilingual/mT5_multilingual_v8KPlus.csv")


frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8]
genDfCombined = pd.concat(frames)
genDfCombined.to_csv('/content/drive/MyDrive/SCDataset/mT5_multilingual_Combined_v.csv')